# Animation example

In [6]:
#@title Mount Google Drive

!pip install omegaconf git+https://github.com/dmarx/keyframed


try:
    from google.colab import drive
    drive.mount('/content/gdrive')
    outputs_path = "/content/gdrive/MyDrive/AI/StabilityAnimations"
    !mkdir -p $outputs_path
except:
    outputs_path = "."
print(f"Animations will be saved to {outputs_path}")

In [7]:
%%capture
#@title Connect to the Stability API

import datetime
import os
import panel as pn
import param
import shutil
import subprocess
import sys

from base64 import b64encode
from IPython import display
from pathlib import Path
from PIL import Image
from tqdm import tqdm


# install Stability SDK for Python
path = Path('stability-sdk')
if path.exists():
    shutil.rmtree(path)
    !pip uninstall -y stability-sdk
!git clone -b dmarx.dev.keyframed --recurse-submodules https://github.com/dmarx/stability-sdk
Path("./stability-sdk/src/stability_sdk/interfaces/__init__.py").touch()
!pip install ./stability-sdk


from stability_sdk.animation import AnimationArgs, Animator
from stability_sdk import client


# GRPC endpoint and engines
GRPC_HOST = "" #@param {type:"string"}
API_KEY = "" #@param {type:"string"}
GENERATE_ENGINE_ID = 'stable-diffusion-v1-5'
INPAINT_ENGINE_ID = 'stable-diffusion-v1-5'
TRANSFORM_ENGINE_ID = 'transform-server-v1'

# Connect to Stability API
stub = client.open_channel(GRPC_HOST, api_key=API_KEY)

In [9]:
# @title Settings

# @markdown Run this cell to reveal the settings UI. After entering values, move on to the next step.

# @markdown To reset values to default, simply re-run this cell.

# @markdown NB: Settings are grouped across several tabs.

show_documentation = True # @param {type:'boolean'}

# #@markdown ####**Resume:**
resume_timestring = "" #@param {type:"string"}


###################

from stability_sdk.animation import (
    BasicSettings,
    AnimationSettings,
    KeyframedSettings,
    CoherenceSettings,
    DepthwarpSettings,
    VideoInputSettings,
    AnimationArgs,
)

args_basic = BasicSettings()
args_anim = AnimationSettings()
args_kf = KeyframedSettings()
args_cohere = CoherenceSettings()
args_depth = DepthwarpSettings()
args_vid = VideoInputSettings()
#args = AnimationArgs() # would be nice if I could link these more easily... there's probably a way.

arg_objs = (
    args_basic,
    args_anim,
    args_kf,
    args_cohere,
    args_depth,
    args_vid,
)


def _show_docs(component):
    cols = []
    for k, v in component.param.objects().items():
        if k == 'name':
          continue
        col = pn.Column(v, v.doc)
        cols.append(col)
    return pn.Column(*cols)

def build(component):
    if show_documentation:
        component = _show_docs(component)
    return pn.Row(component, width=1000)

pn.extension()

#pn.Tabs(*[(a.name[:-5], pn.Row(a.param,width=1000)) for a in arg_objs])
pn.Tabs(*[
    (a.name[:-5], build(a)) for a in arg_objs
])

### Prompts

In [10]:
#animation_prompts = {
#    0: "a painting of a delicious cheeseburger by Tyler Edlin",
#    24: "a painting of the the answer to life the universe and everything by Tyler Edlin",
#}

negative_prompt = ""
negative_prompt_weight = -1.0


from keyframed import Keyframed, to_keyframed
from stability_sdk.animation import Prompt, Prompts
from omegaconf import OmegaConf
from toolz import accumulate
from operator import add
from typing import List

projectName='liil-dicky-round-10'
fpath = f"/content/gdrive/MyDrive/AI/VideoKilledTheRadioStar/{projectName}/storyboard.yaml"
storyboard = OmegaConf.load(fpath)

frames_per_scene = [rec.frames for rec in storyboard.prompt_starts]
tot_frames = sum(frames_per_scene)
keyframes = ([0]+list(accumulate(add,frames_per_scene)))[:-1]

# TO DO: same thing w init images
theme_prompt = "by ralph steadman, extremely detailed, high resolution"
init_images: List[Prompt] = [] 
prompts: List[Prompt] = [] 
for j, k in enumerate(keyframes):
    prompt_text =  storyboard.prompt_starts[j].prompt + f", {theme_prompt}" 
    prompt = Prompt(
        prompt=prompt_text,
        weight_curve=Keyframed(data={k:1})
        )
    init_image = Prompt(
        prompt = storyboard.prompt_starts[j].frame0_fpath,
        weight_curve=Keyframed(data={k:1, (k+1):0})
    )
    if len(prompts):
      prompts[-1].weight_curve[k] = 0
    prompts.append(prompt)
    init_images.append(init_image)


animation_prompts = Prompts(prompts=prompts)
init_images = Prompts(prompts=init_images)

In [12]:
#@title Render the animation

args_d = {}
[args_d.update(a.param.values()) for a in arg_objs]
args=AnimationArgs(**args_d)

# create folder for frames output
if len(resume_timestring):
    out_dir = os.path.join(outputs_path, resume_timestring)
    if not os.path.exists(out_dir):
        raise Exception("Can't resume {resume_timestring} because path {out_dir} doesn't exist. Please make sure the timestring is correct.")
    timestring = resume_timestring
else:
    timestring = datetime.datetime.now().strftime('%Y%m%d%H%M%S')
    out_dir = os.path.join(outputs_path, timestring)
    os.makedirs(out_dir, exist_ok=True)
print(f"Saving animation frames to {out_dir}...")

animator = Animator(
    stub=stub,
    animation_prompts=animation_prompts,
    args=args,
    out_dir=out_dir,    
    negative_prompt=negative_prompt,
    negative_prompt_weight=negative_prompt_weight,
    resume=len(resume_timestring) != 0,
    transform_engine_id=TRANSFORM_ENGINE_ID,
    inpaint_engine_id=INPAINT_ENGINE_ID,
    generate_engine_id=GENERATE_ENGINE_ID,
)

# deal with this later, Prompts type needs to be json serializable i guess
#animator.save_settings(f"{timestring}_settings.txt")

for frame in tqdm(animator.render(), initial=animator.start_frame_idx, total=args.max_frames):
    display.clear_output(wait=True)
    display.display(frame)

In [7]:
#@title Create video from frames
skip_video_for_run_all = False #@param {type: 'boolean'}
fps = 12 #@param {type:"number"}

if skip_video_for_run_all == True:
    print('Skipping video creation, uncheck skip_video_for_run_all if you want to run it')
else:
    image_path = os.path.join(out_dir, "frame_%05d.png")
    mp4_path = os.path.join(out_dir, f"{timestring}.mp4")

    print(f"Compiling animation frames to {mp4_path}...")

    cmd = [
        'ffmpeg',
        '-y',
        '-vcodec', 'png',
        '-r', str(fps),
        '-start_number', str(0),
        '-i', image_path,
        '-c:v', 'libx264',
        '-vf',
        f'fps={fps}',
        '-pix_fmt', 'yuv420p',
        '-crf', '17',
        '-preset', 'veryfast',
        mp4_path
    ]
    process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout, stderr = process.communicate()
    if process.returncode != 0:
        print(stderr)
        raise RuntimeError(stderr)

    mp4 = open(mp4_path,'rb').read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
    display.display( display.HTML(f'<video controls loop><source src="{data_url}" type="video/mp4"></video>') )
